 # Opis notatnika
 Analiza przygotowana w poprzednim kroku została odebrana pozytywnie. W związku z tym, zostaliśmy poproszeni
 o przygotowanie raportu na podstawie danych z roku 2020. Ma być on później  wdrożony przez zespół IT na zewnętrzny
 serwis. Wybór padł na `Dash`.

 Zadanie wykonamy w dwóch krokach:  
 1. Znając wymagania raportu, stworzymy na bazie danych odpowiednie komponenty, które zostaną następnie wykorzystane do wizualizacji.  
 1. Stworzymy raport używając `Dash`.

 Ten notatnik skupia się na odpowiednim przygotowaniu bazy danych pod cele raportowe.

 # Wymagania biznesowe
 Po prezentacji wyników analizy na niezależnym spotkaniu, zostały określone następujące obszary raportowania odsetka opóźnień lotów:  
 1. Wyświetlanie TOP 10 (w sensie najwyższego odsetka opóźnień) lotnisk na podstawie liczby odlotów, wyświetlona ma być również informacja o liczbie przylotów - widok `top_airports_by_departure`.  
 1. Wyświetlenie TOP 10 (w sensie najwyższego odsetka opóźnień) tras lotów. Przy czym istotna dla nas jest kolejność, przykładowo trasa (Warszawa, Paryż) jest inna niż (Paryż, Warszawa). Dodatkowym wymaganiem jest, aby minimalna liczba lotów odbytych na trasie wynosiła co najmniej 10000 przelotów - widok `top_reliability_roads`.  
 1. Porównanie roku 2019 vs. 2020, aby śledzić wpływ COVID na realizację lotów. Interesują nas podejścia:  
     - miesiąc do miesiąca, przykładowo odsetek opoźnień styczeń 2019 vs. odsetek opoźnień styczeń 2020, odsetek opoźnień luty 2019 vs. odsetek opoźnień luty 2020 itd. - widok `year_to_year_comparision`,  
     - dzień do dnia, przykładowo odsetek opoźnień wtorek 2019 vs. odsetek opoźnień wtorek 2020 - widok `day_to_day_comparision`.  
 1. Dzienny, czyli jak danego dnia, globalnie wyglądał wskaźnik opóźnień lotów samolotu, tj. odsetek opóźnień 01-01-2019, odsetek opóźnień 02-01-2019 itd.

 # Podejście techniczne do problemu
 Naszym celem będzie odseparowanie warstwy przygotowania danych (logika raportu) od warstwy prezentacyjnej (wizualizacja).
 Chcemy zapewnić, aby odpowiednie procesy zajmowały się tylko swoimi zadaniami. 
 
 > W tym podejściu warstwa prezentacyjna (wykres/raport) nie implementuje logiki biznesowej w celu przetwarzania danych. Innymi słowy, nie chcemy aby przykładowo agregacja była wykonywana w momencie tworzenia wizualizacji.

 To podejście będzie spójne ze współczesnym sposobem projektowania aplikacji. Ma to też dodatkowy benefit - ze względu na wolumen danych na bazie, nie musimy ich najpierw pobierać - warstwa logiczna je odpowiednio zagreguje i przekaże zdecydowanie mniejszą liczbę wierszy, co przyśpieszy działanie całości.

 # Przygotowanie bazy danych
 Na bazie danych, gdzie umieszczone są już dane, wszystko zawarte jest na schemacie `public`. Ponieważ zgodnie z wymaganiami otrzymujemy nowy obszar wykorzystania danych, stworzymy sobie schemat dedykowany - `reporting`.  
 Dalej stworzymy widoki, które odpowiedzą na zadane wcześniej pytania.

 ## Stworzenie dedykowanego schematu
 W pliku `reporting.sql` tworzymy kwerendę, która stworzy schemat `reporting`.

 # Aktualizacja bazy danych

##### Importujemy potrzebne biblioteki, wczytujemy plik z danymi logowania i nawiązujemy połączenie z bazą danych

In [34]:
import psycopg2
from dotenv import load_dotenv
import os

In [36]:
# Ładujemy zmienne środowiskowe z pliku .env
load_dotenv('DB_pass.env')

True

In [38]:
# Używamy zmiennych środowiskowych do połączenia się z bazą danych
connection = psycopg2.connect(
    dbname=os.getenv("N_DB_NAME"),
    user=os.getenv("DB_USER"),
    password=os.getenv("DB_PASSWORD"),
    host=os.getenv("DB_HOST"),
    port=os.getenv("DB_PORT")
)

cursor = connection.cursor()

 ## Wczytanie pliku `reporting.sql`

In [41]:
# Wczytujemy strukturę raportu z pliku reporting.sql
with open('../sql/reporting.sql', 'r') as file:
    reporting_sql = file.read()

In [43]:
# Dzielimy zawartość pliku na poszczególne komendy zakończone średnikiem
sql_commands = reporting_sql.split(';')

# Dodajemy średnik na końcu każdej komendy, czyścimy tekst z niepotrzbnych znaków białych
sql_commands = [command.strip() + ';' for command in sql_commands]

In [45]:
sql_commands

['CREATE SCHEMA reporting;',
 'CREATE OR REPLACE VIEW reporting.flight AS\nSELECT \n    *,\n    CASE \n        WHEN dep_delay_new > 15 THEN 1\n        ELSE 0\n    END AS is_delayed\nFROM \n    flight\nWHERE \n    cancelled = 0;',
 'CREATE OR REPLACE VIEW reporting.top_airports_by_departure AS\nWITH dep_stat AS (\n    SELECT\n        origin_airport_id AS origin_airport_id,\n        COUNT(is_delayed) AS total_departures,\n        SUM(is_delayed) AS delayed_departures\n    FROM\n        reporting.flight\n    GROUP BY\n        origin_airport_id\n),\narr_stat AS (\n    SELECT\n        dest_airport_id AS dest_airport_id,\n        COUNT(dest_airport_id) AS total_arrivals\n    FROM\n        reporting.flight\n    GROUP BY\n        dest_airport_id\n)\nSELECT\n    d.origin_airport_id,\n    d.total_departures,\n    d.delayed_departures,\n    a.total_arrivals,\n    ROUND((d.delayed_departures * 1.0 / d.total_departures), 4) AS delay_percentage\nFROM\n    dep_stat AS d\nJOIN arr_stat AS a ON d.origi

##### W tym miejscu wczytujemy każdą z kwerend, aby zainicjować raport

In [48]:
try:
    # Iterujemy przez każdą komendę SQL
    for command in sql_commands:
        cursor.execute(command)

    # Jeśli wszystkie komendy zakończą się sukcesem, zatwierdzamy transakcję
    connection.commit()
    print("Wszystkie operacje zakończyły się sukcesem, commit wykonany.")

except Exception as e:
    # W przypadku błędu, wycofujemy wszystkie zmiany
    connection.rollback()
    print(f"Wystąpił błąd: {e}. Wykonano rollback.")

finally:
    # Zamykamy kursor i połączenie w tym miejscu, aby w przypadku wystąpienia błędu połączenie nie zostało otwarte
    cursor.close()
    connection.close()

Wszystkie operacje zakończyły się sukcesem, commit wykonany.


 # Podsumownie
 W tym notatniku stworzyliśmy nowy schemat - `reporting`, którego zadaniem jest przygotowanie naszych danych
 do wizualizacji. Dalsza część pracy będzie polegała na wyświetleniu w wizualnie atrakcyjny sposób danych w interaktywnym raporcie stworzonym
 z pomocą `Dash`.